In [ ]:
using PyCall
sage_all = pyimport("sage.all")

In [ ]:
#using MIPVerify
using Gurobi
using MAT
using IntervalArithmetic
using MIPVerify:prep_data_file
using MIPVerify:read_datasets
using MIPVerify:load_all_binary_images
using MIPVerify:get_image
using MIPVerify:get_label
using MIPVerify:Flatten
using Random
using IntervalArithmetic
using SetRounding
sage_all = pyimport("sage.all")

# 创建 MixedIntegerLinearProgram
p = sage_all.MixedIntegerLinearProgram(solver="ppl",maximization=true)
#p.solver_parameter("timelimit", 60)
#设置输入扰动为0.1


In [ ]:
setprecision(BigFloat, 10)
FLOATMIN = floatmin(Float16)
RATIONALMIN = convert(Rational, BigFloat(floatmin(Float16)))
RATIONAL_ABS = Rational(BigInt(2)^-149)
RATIONAL_REL_UP = 1 + Rational(BigInt(2)^-23)
RATIONAL_REL_LO = 1 - Rational(BigInt(2)^-23)


In [ ]:
typeof(RATIONAL_ABS)

In [ ]:
INTERVAL_REL = Interval(RATIONAL_REL_LO ,RATIONAL_REL_UP )
INTERVAL_ABS = Interval(-RATIONAL_ABS , RATIONAL_ABS)

In [ ]:
struct Interval_Value
    value_up::Rational
    value_lo::Rational
end

In [ ]:
abstract type Layer end
abstract type NeuralNet end

In [ ]:
function convert_to_rational(x::Array{<:AbstractFloat})
    precision = Dict(Float16 => 10, Float32 => 23)
    
    if eltype(x) in [Float16, Float32]
        setprecision(BigFloat, precision[eltype(x)])
    end
    
    return convert.(Rational, BigFloat.(x))
end


In [ ]:
function convert_to_rational_itv(x::Array{<:AbstractFloat})
    precision = Dict(Float16 => 10, Float32 => 23)
    
    if eltype(x) in [Float16, Float32]
        setprecision(BigFloat, precision[eltype(x)])
    end
    
    return Interval.(convert.(Rational, BigFloat.(x)))
end

In [ ]:
struct Ra_Linear{T<:Rational,U<:Rational} <: Layer
    matrix::Array{T,2}
    bias::Array{U,1}

    function Ra_Linear{T,U}(matrix::Array{T,2}, bias::Array{U,1}) where {T<:Rational,U<:Rational}
        (matrix_width, matrix_height) = size(matrix)
        bias_height = length(bias)
        @assert(
            matrix_height == bias_height,
            "Number of output channels in matrix, $matrix_height, does not match number of output channels in bias, $bias_height."
        )
        return new(matrix, bias)
    end

end

function Ra_Linear(matrix::Array{T,2}, bias::Array{U,1}) where {T<:Rational,U<:Rational}
    Ra_Linear{T,U}(matrix, bias)
end

In [ ]:
struct Linear{T<:Real,U<:Real} <: Layer
    matrix::Array{T,2}
    bias::Array{U,1}

    function Linear{T,U}(matrix::Array{T,2}, bias::Array{U,1}) where {T<:Real,U<:Real}
        (matrix_width, matrix_height) = size(matrix)
        bias_height = length(bias)
        @assert(
            matrix_height == bias_height,
            "Number of output channels in matrix, $matrix_height, does not match number of output channels in bias, $bias_height."
        )
        return new(matrix, bias)
    end

end

function Linear(matrix::Array{T,2}, bias::Array{U,1}) where {T<:Real,U<:Real}
    Linear{T,U}(matrix, bias)
end

In [ ]:
#抽象矩阵乘法操作
function matmul(x::Array{T,1},params::Ra_Linear{U,V}) where {T<:Interval,U<:Rational,V<:Rational}
    Weight=transpose(params.matrix)
    (matrix_height, matrix_width) = size(Weight)
    (input_height,) = size(x)
    @assert(
        matrix_width == input_height,
        "Number of values in input, $input_height, does not match number of values, $matrix_height that Linear operates on."
    )

    pre_value = Interval[]
    for i in 1:matrix_height
        tmp_itv = 0
        for j in 1:matrix_width
            #tmp_l = RATIONAL_LO*((RATIONAL_LO*Weight[i,j].lo*x_itv[j].lo-RATIONALMIN) + tmp_l) - RATIONALMIN 
            tmp_itv = INTERVAL_REL*(INTERVAL_REL*(Weight[i,j]*x[j])+INTERVAL_ABS + tmp_itv) +INTERVAL_ABS
        end
        tmp_itv = INTERVAL_REL*(tmp_itv+params.bias[i]) + INTERVAL_ABS
        push!(pre_value,tmp_itv)
    end
    return pre_value

end

In [ ]:
#抽象矩阵乘法操作
function matmul_float(x::Array{T,1},params::Linear{U,V}) where {T<:Real,U<:Real,V<:Real}
    Weight=transpose(params.matrix)
    (matrix_height, matrix_width) = size(Weight)
    (input_height,) = size(x)
    @assert(
        matrix_width == input_height,
        "Number of values in input, $input_height, does not match number of values, $matrix_height that Linear operates on."
    )

    return transpose(params.matrix) * x .+ params.bias

end

In [ ]:
function relu(x_itv::Interval)
    if(x_itv.hi<=0)
        return interval(0)
    else
        return x_itv
    end
end

In [ ]:
function relu(x::Real)
    if(x<=0)
        return 0
    else
        return x
    end
end

In [ ]:
function get_matrix_params(
    param_dict::Dict{String, Any},
    layer_name::String,
    expected_size::NTuple{2, Int};
    matrix_name::String = "weight",
    bias_name::String = "bias",
    data_type::Type = Float32,
    trans2itv::Bool =false,
    debug_test::Bool = false,
    to_double::Bool = false
)::Layer

    param_weight = convert_to_rational(param_dict["$layer_name/$matrix_name"])
    param_bias = convert_to_rational(dropdims(param_dict["$layer_name/$bias_name"], dims=1))
    params = Ra_Linear(param_weight, param_bias)
    return params
end

In [ ]:
function get_matrix_params_float(
    param_dict::Dict{String, Any},
    layer_name::String,
    expected_size::NTuple{2, Int};
    matrix_name::String = "weight",
    bias_name::String = "bias",
    data_type::Type = Float32,
    trans2itv::Bool =false,
    debug_test::Bool = false,
    to_double::Bool = false
)::Layer

    param_weight = param_dict["$layer_name/$matrix_name"]
    param_bias = dropdims(param_dict["$layer_name/$bias_name"], dims=1)
    params = Linear(param_weight, param_bias)
    return params
end

In [ ]:
function get_example_network_params(name::String)
    if name == "MNIST.n1"
        nn = Ra_Linear[]
        param_dict = prep_data_file(joinpath("weights", "mnist"), "n1.mat") |> matread
        fc1 = get_matrix_params(param_dict, "fc1", (784, 40))
        push!(nn,fc1)
        fc2 = get_matrix_params(param_dict, "fc2", (40, 20))
        push!(nn,fc2)
        logits = get_matrix_params(param_dict, "logits", (20, 10))
        push!(nn,logits)
        return nn
    elseif name == "F32MNIST24"
        nn = Ra_Linear[]
        param_dict = prep_data_file(joinpath("weights", "mnist"), "mnist_dnn_fp32.mat") |> matread
        fc1 = get_matrix_params(param_dict, "fc1", (784, 24))
        push!(nn,fc1)
        fc2 = get_matrix_params(param_dict, "fc2", (24, 24))
        push!(nn,fc2)
        logits = get_matrix_params(param_dict, "logits", (24, 10))
        push!(nn,logits)
        return nn
    elseif name == "F32MNIST_INPUT77"
        nn = Ra_Linear[]
        param_dict = prep_data_file(joinpath("weights", "mnist"), "resized77_mnist_dnn_fp32.mat") |> matread
        fc1 = get_matrix_params(param_dict, "fc1", (77, 10))
        push!(nn,fc1)
        fc2 = get_matrix_params(param_dict, "fc2", (10, 10))
        push!(nn,fc2)
        logits = get_matrix_params(param_dict, "logits", (10, 10))
        push!(nn,logits)
        return nn
    end
end



In [ ]:
function get_example_network_params_float(name::String)
    if name == "MNIST.n1"
        nn = Linear[]
        param_dict = prep_data_file(joinpath("weights", "mnist"), "n1.mat") |> matread
        fc1 = get_matrix_params_float(param_dict, "fc1", (784, 40))
        push!(nn,fc1)
        fc2 = get_matrix_params_float(param_dict, "fc2", (40, 20))
        push!(nn,fc2)
        logits = get_matrix_params_float(param_dict, "logits", (20, 10))
        push!(nn,logits)
        return nn
    elseif name == "F32MNIST24"
        nn = Linear[]
        param_dict = prep_data_file(joinpath("weights", "mnist"), "mnist_dnn_fp32.mat") |> matread
        fc1 = get_matrix_params_float(param_dict, "fc1", (784, 24))
        push!(nn,fc1)
        fc2 = get_matrix_params_float(param_dict, "fc2", (24, 24))
        push!(nn,fc2)
        logits = get_matrix_params_float(param_dict, "logits", (24, 10))
        push!(nn,logits)
        return nn
    elseif name == "F32MNIST_INPUT77"
        nn = Linear[]
        param_dict = prep_data_file(joinpath("weights", "mnist"), "resized77_mnist_dnn_fp32.mat") |> matread
        fc1 = get_matrix_params_float(param_dict, "fc1", (77, 10))
        push!(nn,fc1)
        fc2 = get_matrix_params_float(param_dict, "fc2", (10, 10))
        push!(nn,fc2)
        logits = get_matrix_params_float(param_dict, "logits", (10, 10))
        push!(nn,logits)
        return nn
    end
end

In [ ]:
function propagation(x::Vector{Interval{Rational{BigInt}}}, params::Vector{Ra_Linear}) 
    for (i, layer) in enumerate(params)
        x = matmul(x, layer)
        float_x = convert.(Interval{Float64},x)
        #println("\n")
        #println("the res for l$i: $float_x")
        if i < length(params)
            x = relu.(x)
        end
    end
    return x
end


In [ ]:
function propagation_float(x::Vector{<:Real}, params::Vector{Linear}) 
    for (i, layer) in enumerate(params)
        x = matmul_float(x, layer)
        #println("the res for l$i: $x")
        if i < length(params)
            x = relu.(x)
        end
    end
    return x
end

In [ ]:
nn77 = get_example_network_params("F32MNIST_INPUT77")
nn_float77 = get_example_network_params_float("F32MNIST_INPUT77")

In [ ]:
binary_file_path = "/home/aritifact/aritifact_for_cp24ifact_for_cp24/FloatMIPVerify/resized_images/resized_mnist_images77_test.bin"
image_size = (7, 7)
num_images = 10000
images, labels = load_all_binary_images(binary_file_path, image_size, num_images)
index = 100
sample_image = images[index]
sample_image = reshape(images[index], (1, 7, 7, 1))
sample_label = labels[index]
flat_img = Float32.(Flatten(4)(sample_image))
ra_sample_image = convert_to_rational_itv(flat_img)
output = propagation_float(flat_img, nn_float77)
output_itv = propagation(ra_sample_image, nn77)


In [ ]:
using IntervalArithmetic

struct SampleInfo
    index::Int
    output_itv::Vector{Interval}
    output::Vector{Float32}
end

function analyze_samples()
    bounded_num = Int[]
    right_classified = Int[]
    wrong_classified = SampleInfo[]
    misclassified = 0
    
    for index in 1:10000
        try
            sample_image = images[index]
            sample_image = reshape(images[index], (1, 7, 7, 1))
            sample_label = labels[index]
            flat_img = Float32.(Flatten(4)(sample_image))
            ra_sample_image = convert_to_rational_itv(flat_img)
            output = propagation_float(flat_img, nn_float77)
            output_itv = propagation(ra_sample_image, nn77)

            # Check if the predicted label matches the true label
            if argmax(output) != sample_label + 1
                misclassified = misclassified+1
                continue
            end

            # Check if the output is within the computed interval
            @assert all(output .<= sup.(output_itv)) && all(output .>= inf.(output_itv))

            # Update statistics based on classification result
            push!(bounded_num, index)
            if argmax(output_itv) == argmax(output)
                push!(right_classified, index)
            else
                println("Sample itv $index misclassified.")
                sample_wrong = SampleInfo(index, output_itv, output)
                #println(sample_wrong)
                push!(wrong_classified,sample_wrong)
            end
        catch e
            println("Error processing sample $index: $e")
        end
    end

    run_statistics = Dict(
        :bounded_num => bounded_num,
        :right_classified => right_classified,
        :wrong_classified => wrong_classified,
        :misclassified => misclassified
    )
    return run_statistics
end

run_statistics = analyze_samples()

In [ ]:
total = 10000-run_statistics[:misclassified]

In [ ]:
size(run_statistics[:bounded_num])

In [ ]:
nn784 = get_example_network_params("F32MNIST24")
nn_float784 = get_example_network_params_float("F32MNIST24")
mnist = read_datasets("MNIST")

In [ ]:
function analyze_samples()
    bounded_num = Int[]
    right_classified = Int[]
    wrong_classified = SampleInfo[]
    misclassified = 0
    
    for index in 1:10000
        try
            sample_image = get_image(mnist.test.images, index)
            sample_label = get_label(mnist.test.labels, index)
            flat_img = Float32.(Flatten(4)(sample_image))
            ra_sample_image = convert_to_rational_itv(flat_img)
            output = propagation_float(flat_img, nn_float784)
            output_itv = propagation(ra_sample_image, nn784)

            # Check if the predicted label matches the true label
            if argmax(output) != sample_label + 1
                #println("Sample $index misclassified.")
                misclassified = misclassified+1
                continue
            end

            # Check if the output is within the computed interval
            @assert all(output .<= sup.(output_itv)) && all(output .>= inf.(output_itv))

            # Update statistics based on classification result
            push!(bounded_num, index)
            if argmax(output_itv) == argmax(output)
                push!(right_classified, index)
            else
                #println("Sample $index misclassified.")
                sample_wrong = SampleInfo(index, output_itv, output)
                #println(sample_wrong)
                push!(wrong_classified,sample_wrong)
            end
        catch e
            println("Error processing sample $index: $e")
        end
    end

    run_statistics = Dict(
        :bounded_num => bounded_num,
        :right_classified => right_classified,
        :wrong_classified => wrong_classified,
        :misclassified => misclassified
    )
    return run_statistics
end

run_statistics = analyze_samples()


In [ ]:
total = 10000-run_statistics[:misclassified]

In [ ]:
size(run_statistics[:bounded_num])